In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_log_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge

In [2]:
train_path = r"C:\Users\james\J_Data_Lab\kaggle\insurance\data\train.csv"
test_path = r"C:\Users\james\J_Data_Lab\kaggle\insurance\data\test.csv"

In [3]:
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [4]:
def eda_basic_info(df, df_name="DataFrame"):
    print(f"\n===== {df_name} =====")
    print("Head (5 rows):")
    display(df.head())
    
    print("\nInfo:")
    print(df.info())
    
    print("\nDescribe (numeric):")
    display(df.describe())
    
    print("\nNull counts:")
    print(df.isnull().sum())
    
    print("="*40)

In [5]:
eda_basic_info(train, "Train")


===== Train =====
Head (5 rows):


,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0



Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1200000 non-null  int64  
 1   Age                   1181295 non-null  float64
 2   Gender                1200000 non-null  object 
 3   Annual Income         1155051 non-null  float64
 4   Marital Status        1181471 non-null  object 
 5   Number of Dependents  1090328 non-null  float64
 6   Education Level       1200000 non-null  object 
 7   Occupation            841925 non-null   object 
 8   Health Score          1125924 non-null  float64
 9   Location              1200000 non-null  object 
 10  Policy Type           1200000 non-null  object 
 11  Previous Claims       835971 non-null   float64
 12  Vehicle Age           1199994 non-null  float64
 13  Credit Score          1062118 non-null  float64
 14  Insurance Duration    11999

,id,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount
count,1.200000e+06,1.181295e+06,1.155051e+06,1.090328e+06,1.125924e+06,835971.000000,1.199994e+06,1.062118e+06,1.199999e+06,1.200000e+06
mean,5.999995e+05,4.114556e+01,3.274522e+04,2.009934e+00,2.561391e+01,1.002689,9.569889e+00,5.929244e+02,5.018219e+00,1.102545e+03
std,3.464103e+05,1.353995e+01,3.217951e+04,1.417338e+00,1.220346e+01,0.982840,5.776189e+00,1.499819e+02,2.594331e+00,8.649989e+02
min,0.000000e+00,1.800000e+01,1.000000e+00,0.000000e+00,2.012237e+00,0.000000,0.000000e+00,3.000000e+02,1.000000e+00,2.000000e+01
25%,2.999998e+05,3.000000e+01,8.001000e+03,1.000000e+00,1.591896e+01,0.000000,5.000000e+00,4.680000e+02,3.000000e+00,5.140000e+02
50%,5.999995e+05,4.100000e+01,2.391100e+04,2.000000e+00,2.457865e+01,1.000000,1.000000e+01,5.950000e+02,5.000000e+00,8.720000e+02
75%,8.999992e+05,5.300000e+01,4.463400e+04,3.000000e+00,3.452721e+01,2.000000,1.500000e+01,7.210000e+02,7.000000e+00,1.509000e+03
max,1.199999e+06,6.400000e+01,1.499970e+05,4.000000e+00,5.897591e+01,9.000000,1.900000e+01,8.490000e+02,9.000000e+00,4.999000e+03



Null counts:
id                           0
Age                      18705
Gender                       0
Annual Income            44949
Marital Status           18529
Number of Dependents    109672
Education Level              0
Occupation              358075
Health Score             74076
Location                     0
Policy Type                  0
Previous Claims         364029
Vehicle Age                  6
Credit Score            137882
Insurance Duration           1
Policy Start Date            0
Customer Feedback        77824
Smoking Status               0
Exercise Frequency           0
Property Type                0
Premium Amount               0
dtype: int64


In [6]:
eda_basic_info(test, "Test")


===== Test =====
Head (5 rows):


,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type
0,1200000,28.0,Female,2310.0,NaN,4.0,Bachelor's,Self-Employed,7.657981,Rural,Basic,NaN,19.0,NaN,1.0,2023-06-04 15:21:39.245086,Poor,Yes,Weekly,House
1,1200001,31.0,Female,126031.0,Married,2.0,Master's,Self-Employed,13.381379,Suburban,Premium,NaN,14.0,372.0,8.0,2024-04-22 15:21:39.224915,Good,Yes,Rarely,Apartment
2,1200002,47.0,Female,17092.0,Divorced,0.0,PhD,Unemployed,24.354527,Urban,Comprehensive,NaN,16.0,819.0,9.0,2023-04-05 15:21:39.134960,Average,Yes,Monthly,Condo
3,1200003,28.0,Female,30424.0,Divorced,3.0,PhD,Self-Employed,5.136225,Suburban,Comprehensive,1.0,3.0,770.0,5.0,2023-10-25 15:21:39.134960,Poor,Yes,Daily,House
4,1200004,24.0,Male,10863.0,Divorced,2.0,High School,Unemployed,11.844155,Suburban,Premium,NaN,14.0,755.0,7.0,2021-11-26 15:21:39.259788,Average,No,Weekly,House



Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    800000 non-null  int64  
 1   Age                   787511 non-null  float64
 2   Gender                800000 non-null  object 
 3   Annual Income         770140 non-null  float64
 4   Marital Status        787664 non-null  object 
 5   Number of Dependents  726870 non-null  float64
 6   Education Level       800000 non-null  object 
 7   Occupation            560875 non-null  object 
 8   Health Score          750551 non-null  float64
 9   Location              800000 non-null  object 
 10  Policy Type           800000 non-null  object 
 11  Previous Claims       557198 non-null  float64
 12  Vehicle Age           799997 non-null  float64
 13  Credit Score          708549 non-null  float64
 14  Insurance Duration    799998 non-null  float6

,id,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration
count,8.000000e+05,787511.000000,770140.000000,726870.000000,750551.000000,557198.000000,799997.000000,708549.000000,799998.000000
mean,1.600000e+06,41.136440,32803.871471,2.009337,25.613036,1.004873,9.571891,592.904749,5.018949
std,2.309403e+05,13.537829,32201.063749,1.415241,12.206882,0.982803,5.772200,150.116374,2.593759
min,1.200000e+06,18.000000,2.000000,0.000000,1.646561,0.000000,0.000000,300.000000,1.000000
25%,1.400000e+06,30.000000,8048.000000,1.000000,15.917353,0.000000,5.000000,468.000000,3.000000
50%,1.600000e+06,41.000000,23981.000000,2.000000,24.580164,1.000000,10.000000,595.000000,5.000000
75%,1.799999e+06,53.000000,44660.000000,3.000000,34.517766,2.000000,15.000000,721.000000,7.000000
max,1.999999e+06,64.000000,149997.000000,4.000000,57.957351,9.000000,19.000000,849.000000,9.000000



Null counts:
id                           0
Age                      12489
Gender                       0
Annual Income            29860
Marital Status           12336
Number of Dependents     73130
Education Level              0
Occupation              239125
Health Score             49449
Location                     0
Policy Type                  0
Previous Claims         242802
Vehicle Age                  3
Credit Score             91451
Insurance Duration           2
Policy Start Date            0
Customer Feedback        52276
Smoking Status               0
Exercise Frequency           0
Property Type                0
dtype: int64


In [8]:
def preprocess_data(df, is_train=True):
    
    # df['Age'].fillna(df['Age'].median(), inplace=True)
    # df['Annual Income'].fillna(df['Annual Income'].median(), inplace=True)
    # df['Number of Dependents'].fillna(df['Number of Dependents'].median(), inplace=True)
    # df['Health Score'].fillna(df['Health Score'].median(), inplace=True)
    # df['Previous Claims'].fillna(0, inplace=True)
    # df['Vehicle Age'].fillna(df['Vehicle Age'].median(), inplace=True)
    # df['Credit Score'].fillna(df['Credit Score'].median(), inplace=True)
    # df['Insurance Duration'].fillna(df['Insurance Duration'].median(), inplace=True)

    
    # df['Marital Status'].fillna(df['Marital Status'].mode()[0], inplace=True)
    # df['Occupation'].fillna('Unknown', inplace=True)
    # df['Customer Feedback'].fillna('No Feedback', inplace=True)

    
    df['Missing_Occupation'] = df['Occupation'].isna().astype(int)
    df['Missing_Credit_Score'] = df['Credit Score'].isna().astype(int)
    df['Policy Start Date'] = pd.to_datetime(df['Policy Start Date'])
    df['Policy Year'] = df['Policy Start Date'].dt.year
    df['Policy Month'] = df['Policy Start Date'].dt.month
    df['Policy Day'] = df['Policy Start Date'].dt.day
    df['Policy Weekday'] = df['Policy Start Date'].dt.weekday

    label_cols = ['Gender', 'Marital Status', 'Education Level', 
                  'Occupation', 'Location', 'Policy Type', 
                  'Customer Feedback', 'Smoking Status', 'Exercise Frequency', 'Property Type']

    for col in label_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])

    df = pd.get_dummies(df, columns=['Property Type'], drop_first=True)

    #if is_train:
     #   drop_cols = ['id', 'Policy Start Date']
    #else:
     #   drop_cols = ['id', 'Policy Start Date']

    #df.drop(columns=drop_cols, inplace=True)
    
    return df

In [9]:
train = preprocess_data(train, is_train=True)
test = preprocess_data(test, is_train=False)

In [10]:
# RMSLE 
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

# modeling
def train_and_evaluate(model_name, X_train, y_train, X_val, y_val):
    # select model
    if model_name == 'xgb':
        model = XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=6, random_state=42)
    elif model_name == 'lgb':
        model = LGBMRegressor(n_estimators=500, learning_rate=0.05, max_depth=6, random_state=42)
    elif model_name == 'cat':
        model = CatBoostRegressor(iterations=500, learning_rate=0.05, depth=6, random_state=42, verbose=0)
    else:
        raise ValueError("Value Error.")
    
    # train
    model.fit(X_train, y_train)
    
    # predict
    preds = model.predict(X_val)
    
    # evaluate
    score = rmsle(y_val, preds)
    print(f"{model_name} RMSLE: {score}")
    
    return model, preds

In [11]:
train['log_premium'] = np.log1p(train['Premium Amount'])
drop_cols = ['id', 'Policy Start Date', 'Premium Amount']
train = train.drop(columns=drop_cols)

# split
X = train.drop('log_premium', axis=1)
y = train['log_premium']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# evaluate
xgb_model, xgb_preds = train_and_evaluate('xgb', X_train, y_train, X_val, y_val)
lgb_model, lgb_preds = train_and_evaluate('lgb', X_train, y_train, X_val, y_val)
cat_model, cat_preds = train_and_evaluate('cat', X_train, y_train, X_val, y_val)

xgb RMSLE: 0.15843615175306353
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.025918 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 953
[LightGBM] [Info] Number of data points in the train set: 960000, number of used features: 25
[LightGBM] [Info] Start training from score 6.593848
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
lgb RMSLE: 0.15846762156994398
cat RMSLE: 0.15861767536705948


In [ ]:
# staking
stacking_model = StackingRegressor(
    estimators=[
        ('xgb', XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=6, random_state=42)),
        ('lgb', LGBMRegressor(n_estimators=500, learning_rate=0.05, max_depth=6, random_state=42)),
        ('cat', CatBoostRegressor(iterations=500, learning_rate=0.05, depth=6, random_state=42, verbose=0))
    ],
    final_estimator=Ridge(alpha=1.0)  # Ridge
)

stacking_model.fit(X_train, y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.045644 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 953
[LightGBM] [Info] Number of data points in the train set: 960000, number of used features: 25
[LightGBM] [Info] Start training from score 6.593848
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.019813 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 954
[LightGBM] [Info] Number of data points in the train set: 768000, number of used features: 25
[LightGBM] [Info] Start training from score 6.594524
[LightGBM] [Warni

In [12]:
stack_preds = stacking_model.predict(X_val)

# RMSLE
stack_rmsle = rmsle(y_val, stack_preds)
print(f"Stacking Model RMSLE: {stack_rmsle}")

# Stacking Model RMSLE: 0.1586733286192902
# Stacking Model RMSLE: 0.15839910080580086

Stacking Model RMSLE: 0.15839910080580086


In [13]:
#test_id = test['id']
#test = test.drop(columns=['id', 'Policy Start Date'])

test = test[X_train.columns]
test_preds = stacking_model.predict(test)
test_preds = np.expm1(test_preds)

# submission
submission = pd.DataFrame({
    'id': test_id,  # 전처리 전 id 사용
    'Premium Amount': test_preds
})
submission.to_csv('submission_stacking_20241231.csv', index=False)